In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import os
from tensorflow.keras import regularizers,  layers
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import (Input, Conv1D, Conv2D, BatchNormalization, Activation, Add,
                                     MaxPooling2D, GlobalAveragePooling2D, Dense, Reshape, MaxPooling1D, Flatten, Dropout, LSTM, TimeDistributed, Bidirectional,SeparableConv2D)

# Importar datos para  entrenamiento

In [17]:
# Rutas de los datos procesados
TRAIN_PATH = "../data_train/processed_audio/train.npz"
TEST_PATH = "../data_train/processed_audio/test.npz"

# Cargar datos
data_train = np.load(TRAIN_PATH)
data_test = np.load(TEST_PATH)

X_train, y_train = data_train["X"], data_train["y"]
X_test, y_test = data_test["X"], data_test["y"]

# Verificar forma de los datos
print("Datos cargados correctamente:")
print(f"X_train: {X_train.shape}, y_train: {y_train.shape}")
print(f"X_test: {X_test.shape}, y_test: {y_test.shape}")

# Imprimir distribución de etiquetas en el conjunto de entrenamiento
labels_train = np.argmax(y_train, axis=1)
unique_labels_train, counts_train = np.unique(labels_train, return_counts=True)
print("\nDistribución de etiquetas en y_train:")
for label, count in zip(unique_labels_train, counts_train):
    print(f"Etiqueta {label}: {count} ejemplos")

# Imprimir distribución de etiquetas en el conjunto de prueba
labels_test = np.argmax(y_test, axis=1)
unique_labels_test, counts_test = np.unique(labels_test, return_counts=True)
print("\nDistribución de etiquetas en y_test:")
for label, count in zip(unique_labels_test, counts_test):
    print(f"Etiqueta {label}: {count} ejemplos")

# Crear `tf.data.Dataset` para optimizar el entrenamiento
BATCH_SIZE = 32
AUTOTUNE = tf.data.AUTOTUNE  # Ajuste automático del prefetching

train_dataset = (
    tf.data.Dataset.from_tensor_slices((X_train, y_train))
    .shuffle(len(X_train))  # Mezclar datos
    .batch(BATCH_SIZE)
    .prefetch(AUTOTUNE)
)

test_dataset = (
    tf.data.Dataset.from_tensor_slices((X_test, y_test))
    .batch(BATCH_SIZE)
    .prefetch(AUTOTUNE)
)

print("\nDatasets creados y listos para entrenar.")


Datos cargados correctamente:
X_train: (2014, 16000), y_train: (2014, 2)
X_test: (504, 16000), y_test: (504, 2)

Distribución de etiquetas en y_train:
Etiqueta 0: 1007 ejemplos
Etiqueta 1: 1007 ejemplos

Distribución de etiquetas en y_test:
Etiqueta 0: 252 ejemplos
Etiqueta 1: 252 ejemplos

Datasets creados y listos para entrenar.


## Modelo: NN simple


In [3]:
# Parámetros
num_features = X_train.shape[1]  # Longitud de entrada (1 segundo de audio a 16kHz)
num_classes = y_train.shape[1]   # 2 categorías (cry, noise)


# Definir el modelo
model = Sequential([
    Input(shape=(num_features,)),  # Capa de entrada
    Dense(32, activation='relu'), # Capa oculta
    Dense(16, activation='relu'), # Capa oculta
    Dense(num_classes, activation='softmax')  # Capa de salida
])

# Compilar el modelo
model.compile(optimizer="adam",
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Mostrar resumen del modelo
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 32)                512032    
                                                                 
 dense_1 (Dense)             (None, 16)                528       
                                                                 
 dense_2 (Dense)             (None, 2)                 34        
                                                                 
Total params: 512,594
Trainable params: 512,594
Non-trainable params: 0
_________________________________________________________________


In [ ]:
es = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True) # Detener si no hay mejora en la validación tras 3 épocas. Se  queda con los mejores pesos.
mc = ModelCheckpoint("../models/not_compressed_audio/model_NN.h5", save_best_only=True, monitor='val_loss') # Guardar el mejor modelo basado en la validación.

history = model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=30,
    batch_size=32,
    callbacks=[es, mc]
)

Epoch 1/30
63/63 [==============================] - 1s 11ms/step - loss: 0.6359 - accuracy: 0.6216 - val_loss: 0.5582 - val_accuracy: 0.7222
Epoch 2/30
63/63 [==============================] - 0s 7ms/step - loss: 0.2659 - accuracy: 0.9394 - val_loss: 0.5417 - val_accuracy: 0.7639
Epoch 3/30
63/63 [==============================] - 0s 7ms/step - loss: 0.0888 - accuracy: 0.9886 - val_loss: 0.6717 - val_accuracy: 0.7421
Epoch 4/30
63/63 [==============================] - 0s 6ms/step - loss: 0.0418 - accuracy: 0.9926 - val_loss: 0.8876 - val_accuracy: 0.6825
Epoch 5/30
63/63 [==============================] - 0s 7ms/step - loss: 0.0348 - accuracy: 0.9945 - val_loss: 1.0809 - val_accuracy: 0.6964


## Modelo: NN avanzada

In [ ]:
num_features = X_train.shape[1]
num_classes = y_train.shape[1]

model = Sequential([
    Input(shape=(num_features,)),

    Dense(64, activation='relu'),
    BatchNormalization(),
    Dropout(0.3),

    Dense(32, activation='relu'),
    BatchNormalization(),
    Dropout(0.3),

    Dense(16, activation='relu'),
    BatchNormalization(),
    Dropout(0.3),

    Dense(8, activation='relu'),
    BatchNormalization(),
    Dropout(0.3),

    Dense(num_classes, activation='softmax')
])

model.compile(optimizer="adam",
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 64)                1024064   
                                                                 
 batch_normalization (BatchN  (None, 64)               256       
 ormalization)                                                   
                                                                 
 dropout (Dropout)           (None, 64)                0         
                                                                 
 dense_4 (Dense)             (None, 32)                2080      
                                                                 
 batch_normalization_1 (Batc  (None, 32)               128       
 hNormalization)                                                 
                                                                 
 dropout_1 (Dropout)         (None, 32)               

In [6]:
es = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
mc = ModelCheckpoint("../models/not_compressed_audio/model_NNA.h5", save_best_only=True, monitor='val_loss')

history = model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=30,
    batch_size=32,
    callbacks=[es, mc]
)

Epoch 1/30
63/63 [==============================] - 3s 17ms/step - loss: 0.9165 - accuracy: 0.5268 - val_loss: 0.6687 - val_accuracy: 0.5913
Epoch 2/30
63/63 [==============================] - 1s 12ms/step - loss: 0.7436 - accuracy: 0.5824 - val_loss: 0.6565 - val_accuracy: 0.5992
Epoch 3/30
63/63 [==============================] - 1s 12ms/step - loss: 0.6877 - accuracy: 0.6107 - val_loss: 0.6436 - val_accuracy: 0.6230
Epoch 4/30
63/63 [==============================] - 1s 13ms/step - loss: 0.6273 - accuracy: 0.6514 - val_loss: 0.6165 - val_accuracy: 0.7044
Epoch 5/30
63/63 [==============================] - 1s 13ms/step - loss: 0.5935 - accuracy: 0.6832 - val_loss: 0.5881 - val_accuracy: 0.6984
Epoch 6/30
63/63 [==============================] - 1s 12ms/step - loss: 0.5510 - accuracy: 0.7274 - val_loss: 0.5794 - val_accuracy: 0.6825
Epoch 7/30
63/63 [==============================] - 1s 12ms/step - loss: 0.4986 - accuracy: 0.7656 - val_loss: 0.5698 - val_accuracy: 0.6825
Epoch 8/30
63

## Modelo: CNN simple

In [18]:
num_features = X_train.shape[1]
num_classes = y_train.shape[1]

model = Sequential([
    Reshape((num_features, 1), input_shape=(num_features,), name="add_channel"),

    Conv1D(16, kernel_size=9, strides=4, activation='relu', name="conv1"),
    MaxPooling1D(pool_size=4, name="pool1"),

    Conv1D(8, kernel_size=9, strides=4, activation='relu', name="conv2"),
    MaxPooling1D(pool_size=4, name="pool2"),

    Flatten(name="flatten"),
    Dense(16, activation='relu', name="dense1"),

    Dense(2, activation='softmax', name="output")
])

model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

model.summary()


Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 add_channel (Reshape)       (None, 16000, 1)          0         
                                                                 
 conv1 (Conv1D)              (None, 3998, 16)          160       
                                                                 
 pool1 (MaxPooling1D)        (None, 999, 16)           0         
                                                                 
 conv2 (Conv1D)              (None, 248, 8)            1160      
                                                                 
 pool2 (MaxPooling1D)        (None, 62, 8)             0         
                                                                 
 flatten (Flatten)           (None, 496)               0         
                                                                 
 dense1 (Dense)              (None, 16)               

In [19]:
es = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
mc = ModelCheckpoint("../models/not_compressed_audio/model_CNN1.h5", save_best_only=True, monitor='val_loss')

history = model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=30,
    batch_size=32,
    callbacks=[es, mc]
)

Epoch 1/30
63/63 [==============================] - 4s 47ms/step - loss: 0.6271 - accuracy: 0.6693 - val_loss: 0.5134 - val_accuracy: 0.7738
Epoch 2/30
63/63 [==============================] - 3s 41ms/step - loss: 0.4931 - accuracy: 0.7915 - val_loss: 0.4377 - val_accuracy: 0.8234
Epoch 3/30
63/63 [==============================] - 3s 44ms/step - loss: 0.4524 - accuracy: 0.8128 - val_loss: 0.4982 - val_accuracy: 0.7857
Epoch 4/30
63/63 [==============================] - 3s 39ms/step - loss: 0.4204 - accuracy: 0.8292 - val_loss: 0.4002 - val_accuracy: 0.8373
Epoch 5/30
63/63 [==============================] - 3s 42ms/step - loss: 0.3893 - accuracy: 0.8530 - val_loss: 0.3917 - val_accuracy: 0.8433
Epoch 6/30
63/63 [==============================] - 2s 40ms/step - loss: 0.3679 - accuracy: 0.8525 - val_loss: 0.3761 - val_accuracy: 0.8532
Epoch 7/30
63/63 [==============================] - 3s 40ms/step - loss: 0.3533 - accuracy: 0.8570 - val_loss: 0.3764 - val_accuracy: 0.8571
Epoch 8/30
63

## Modelo: CNN avanzada

In [9]:
num_features = X_train.shape[1]
num_classes  = y_train.shape[1]


model = Sequential([
    Reshape((num_features, 1), input_shape=(num_features,), name="add_channel"),

    Conv1D(64, 9, activation='relu', padding='same'),
    MaxPooling1D(pool_size=4, name="pool1"),

    Conv1D(32, 9, activation='relu', padding='same'),
    MaxPooling1D(pool_size=4, name="pool2"),

    Conv1D(16, 9, activation='relu', padding='same'),
    MaxPooling1D(pool_size=4, name="pool3"),

    Flatten(name="flatten"),
    Dense(64, activation='relu', name="dense1"),

    Dense(num_classes, activation='softmax', name="output")
])

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model.summary()


Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 add_channel (Reshape)       (None, 16000, 1)          0         
                                                                 
 conv1d (Conv1D)             (None, 16000, 64)         640       
                                                                 
 pool1 (MaxPooling1D)        (None, 4000, 64)          0         
                                                                 
 conv1d_1 (Conv1D)           (None, 4000, 32)          18464     
                                                                 
 pool2 (MaxPooling1D)        (None, 1000, 32)          0         
                                                                 
 conv1d_2 (Conv1D)           (None, 1000, 16)          4624      
                                                                 
 pool3 (MaxPooling1D)        (None, 250, 16)          

In [10]:
es = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
mc = ModelCheckpoint("../models/not_compressed_audio/model_CNNA.h5", save_best_only=True, monitor='val_loss')

history = model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=30,
    batch_size=32,
    callbacks=[es, mc]
)

Epoch 1/30
63/63 [==============================] - 35s 548ms/step - loss: 0.5529 - accuracy: 0.7284 - val_loss: 0.4410 - val_accuracy: 0.8155
Epoch 2/30
63/63 [==============================] - 34s 538ms/step - loss: 0.3990 - accuracy: 0.8357 - val_loss: 0.3696 - val_accuracy: 0.8591
Epoch 3/30
63/63 [==============================] - 34s 540ms/step - loss: 0.3327 - accuracy: 0.8674 - val_loss: 0.3368 - val_accuracy: 0.8690
Epoch 4/30
63/63 [==============================] - 34s 542ms/step - loss: 0.2601 - accuracy: 0.8997 - val_loss: 0.3818 - val_accuracy: 0.8433
Epoch 5/30
63/63 [==============================] - 33s 524ms/step - loss: 0.2178 - accuracy: 0.9156 - val_loss: 0.2930 - val_accuracy: 0.8948
Epoch 6/30
63/63 [==============================] - 34s 535ms/step - loss: 0.1416 - accuracy: 0.9518 - val_loss: 0.2960 - val_accuracy: 0.9087
Epoch 7/30
63/63 [==============================] - 33s 527ms/step - loss: 0.1168 - accuracy: 0.9588 - val_loss: 0.2969 - val_accuracy: 0.9187

## Modelos para MFCC

In [11]:
# Rutas de los datos procesados
TRAIN_PATH = "../data_train/processed_MFCC/train.npz"
TEST_PATH = "../data_train/processed_MFCC/test.npz"

# Cargar datos procesados
data_train = np.load(TRAIN_PATH)
data_test = np.load(TEST_PATH)

X_train, y_train = data_train["X"], data_train["y"]
X_test, y_test = data_test["X"], data_test["y"]

# Verificar forma de los datos
print("Datos cargados correctamente:")
print(f"X_train: {X_train.shape}, y_train: {y_train.shape}")
print(f"X_test: {X_test.shape}, y_test: {y_test.shape}")

# Imprimir distribución de etiquetas en el conjunto de entrenamiento
labels_train = np.argmax(y_train, axis=1)
unique_labels_train, counts_train = np.unique(labels_train, return_counts=True)
print("\nDistribución de etiquetas en y_train:")
for label, count in zip(unique_labels_train, counts_train):
    print(f"Etiqueta {label}: {count} ejemplos")

# Imprimir distribución de etiquetas en el conjunto de prueba
labels_test = np.argmax(y_test, axis=1)
unique_labels_test, counts_test = np.unique(labels_test, return_counts=True)
print("\nDistribución de etiquetas en y_test:")
for label, count in zip(unique_labels_test, counts_test):
    print(f"Etiqueta {label}: {count} ejemplos")


BATCH_SIZE = 32
AUTOTUNE = tf.data.AUTOTUNE

train_dataset = (
    tf.data.Dataset.from_tensor_slices((tf.cast(X_train, tf.float32), y_train))
    .cache()
    .shuffle(buffer_size=len(X_train))
    .batch(BATCH_SIZE)
    .prefetch(AUTOTUNE)
)

test_dataset = (
    tf.data.Dataset.from_tensor_slices((tf.cast(X_test, tf.float32), y_test))
    .cache()
    .batch(BATCH_SIZE)
    .prefetch(AUTOTUNE)
)

print("Datasets listos para entrenar.")


Datos cargados correctamente:
X_train: (2012, 49, 13), y_train: (2012, 2)
X_test: (504, 49, 13), y_test: (504, 2)

Distribución de etiquetas en y_train:
Etiqueta 0: 1005 ejemplos
Etiqueta 1: 1007 ejemplos

Distribución de etiquetas en y_test:
Etiqueta 0: 252 ejemplos
Etiqueta 1: 252 ejemplos
Datasets listos para entrenar.


In [12]:
input_shape = X_train.shape[1:]
num_classes = y_train.shape[1]

model = Sequential([

        Input(shape=input_shape),
        Reshape(input_shape + (1,)),


        Conv2D(32, kernel_size=(3, 3), activation='relu', padding='same'),
        BatchNormalization(),
        MaxPooling2D(pool_size=(2, 2)),
        Dropout(0.3),

        Conv2D(64, kernel_size=(3, 3), activation='relu', padding='same'),
        BatchNormalization(),
        MaxPooling2D(pool_size=(2, 2)),
        Dropout(0.4),

        Flatten(),
        Dense(64, activation='relu'),
        Dense(num_classes, activation='softmax', kernel_regularizer=regularizers.l2(0.001))
])


model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])


model.summary()



Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 reshape (Reshape)           (None, 49, 13, 1)         0         
                                                                 
 conv2d (Conv2D)             (None, 49, 13, 32)        320       
                                                                 
 batch_normalization_4 (Batc  (None, 49, 13, 32)       128       
 hNormalization)                                                 
                                                                 
 max_pooling2d (MaxPooling2D  (None, 24, 6, 32)        0         
 )                                                               
                                                                 
 dropout_4 (Dropout)         (None, 24, 6, 32)         0         
                                                                 
 conv2d_1 (Conv2D)           (None, 24, 6, 64)        

In [13]:
es = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
mc = ModelCheckpoint("../models/not_compressed_MFCC/model_CNN.h5", save_best_only=True, monitor='val_loss')

history = model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=30,
    batch_size=32,
    callbacks=[es, mc]
)

Epoch 1/30
63/63 [==============================] - 3s 35ms/step - loss: 0.6030 - accuracy: 0.7992 - val_loss: 0.5648 - val_accuracy: 0.6706
Epoch 2/30
63/63 [==============================] - 2s 33ms/step - loss: 0.2850 - accuracy: 0.8733 - val_loss: 0.2810 - val_accuracy: 0.8790
Epoch 3/30
63/63 [==============================] - 2s 33ms/step - loss: 0.2238 - accuracy: 0.9026 - val_loss: 0.2326 - val_accuracy: 0.8988
Epoch 4/30
63/63 [==============================] - 2s 34ms/step - loss: 0.2009 - accuracy: 0.9225 - val_loss: 0.1842 - val_accuracy: 0.9187
Epoch 5/30
63/63 [==============================] - 2s 32ms/step - loss: 0.1628 - accuracy: 0.9384 - val_loss: 0.2075 - val_accuracy: 0.9206
Epoch 6/30
63/63 [==============================] - 2s 31ms/step - loss: 0.1748 - accuracy: 0.9334 - val_loss: 0.2313 - val_accuracy: 0.9087
Epoch 7/30
63/63 [==============================] - 2s 31ms/step - loss: 0.1405 - accuracy: 0.9493 - val_loss: 0.2049 - val_accuracy: 0.9266


In [14]:
input_shape = X_train.shape[1:]
num_classes = y_train.shape[1]

model = Sequential([
        Input(shape=input_shape),
        Reshape(input_shape + (1,)),

        Conv2D(16, kernel_size=(3, 3), activation='relu', padding='same'),
        BatchNormalization(),
        MaxPooling2D(pool_size=(2, 2)),
        Dropout(0.3),


        Conv2D(32, kernel_size=(3, 3), activation='relu', padding='same'),
        BatchNormalization(),
        MaxPooling2D(pool_size=(2, 2)),
        Dropout(0.3),

        Conv2D(64, kernel_size=(3, 3), activation='relu', padding='same'),
        BatchNormalization(),
        MaxPooling2D(pool_size=(2, 2)),
        Dropout(0.4),

        Flatten(),
        Dense(64, activation='relu'),
        Dense(32, activation='relu'),
        Dense(num_classes, activation='softmax', kernel_regularizer=regularizers.l2(0.001))
])


model.compile(optimizer="adam",
              loss='categorical_crossentropy',
              metrics=['accuracy'])


model.summary()



Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 reshape_1 (Reshape)         (None, 49, 13, 1)         0         
                                                                 
 conv2d_2 (Conv2D)           (None, 49, 13, 16)        160       
                                                                 
 batch_normalization_6 (Batc  (None, 49, 13, 16)       64        
 hNormalization)                                                 
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 24, 6, 16)        0         
 2D)                                                             
                                                                 
 dropout_6 (Dropout)         (None, 24, 6, 16)         0         
                                                                 
 conv2d_3 (Conv2D)           (None, 24, 6, 32)        

In [15]:
es = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
mc = ModelCheckpoint("../models/not_compressed_MFCC/model_CNNA.h5", save_best_only=True, monitor='val_loss')

history = model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=30,
    batch_size=32,
    callbacks=[es, mc]
)

Epoch 1/30
63/63 [==============================] - 3s 28ms/step - loss: 0.5610 - accuracy: 0.7475 - val_loss: 0.4754 - val_accuracy: 0.7778
Epoch 2/30
63/63 [==============================] - 2s 26ms/step - loss: 0.3965 - accuracy: 0.8270 - val_loss: 0.3617 - val_accuracy: 0.8552
Epoch 3/30
63/63 [==============================] - 2s 24ms/step - loss: 0.3422 - accuracy: 0.8514 - val_loss: 0.3321 - val_accuracy: 0.8591
Epoch 4/30
63/63 [==============================] - 2s 24ms/step - loss: 0.3054 - accuracy: 0.8658 - val_loss: 0.3234 - val_accuracy: 0.8571
Epoch 5/30
63/63 [==============================] - 1s 23ms/step - loss: 0.2809 - accuracy: 0.8807 - val_loss: 0.3353 - val_accuracy: 0.8552
Epoch 6/30
63/63 [==============================] - 1s 23ms/step - loss: 0.2638 - accuracy: 0.8842 - val_loss: 0.3048 - val_accuracy: 0.8651
Epoch 7/30
63/63 [==============================] - 2s 24ms/step - loss: 0.2388 - accuracy: 0.9021 - val_loss: 0.2809 - val_accuracy: 0.8869
Epoch 8/30
63